# Energy Storage Dispatch Algorithm
In this notebook, we'll lay out the algorithm for our **energy storage dispatch** in commercial buildings.

**Key variables:**
* Factor A
* Factor B
* Factor C

**Parameter:**
* Charge Level(?)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./path/to/csv/here.csv')
df.head()